<h1>Delete ContraxSuite user</h1>

Navigate to the last cell to:
- list ContraxsuiteUsers
- show user detail
- delete specific user

In [6]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# setup django
import os
import sys
sys.path.append('../../')
os.environ["DJANGO_SETTINGS_MODULE"] = "settings"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings")
import django
django.setup()

from django.conf import settings
from django.db.models import *

In [7]:
from typing import Optional, List
from apps.users.models import User
from django.db.models import Q
import datetime
from django.utils.timezone import make_aware
import pandas as pd
from IPython.display import display


def show_users_activity(
    id_filter: Optional[List[int]] = None,
    name: Optional[str] = None,
    sort_field: Optional[str] = 'last_login',
    sort_order: str = 'desc'):
    
    sort_field = sort_field or 'last_login'
    
    query = User.objects.all()
    q = Q()
                        
    if id_filter:
        q = q | Q(id__in=id_filter)
    if name:
        q = q | Q(name__startswith=name) | Q(username__startswith=name)
        q = q | Q(first_name__startswith=name) | Q(last_name__startswith=name)
    if q:
        query = query.filter(q)
                        
    users = list(query)
    
    user_data = [{'id': u.pk, 'name': u.name or u.username, 'full name': u.get_full_name(),
                  'last_login': u.last_login, 'projects': len(u.user_projects)} 
                 for u in users]
    
    if sort_field == 'last_login':
        def_date = datetime.datetime(1980, 1, 1)
        def_date = make_aware(def_date)    
        user_data.sort(key=lambda u: u['last_login'] or def_date, reverse=sort_order == 'desc')
    else:
        user_data.sort(key=lambda u: u[sort_field], reverse=sort_order == 'desc')
        
    for u in user_data:
        u['last_login'] = u['last_login'].strftime('%Y-%m-%d %H:%M') if u['last_login'] else 'never'
    
    
    df = pd.DataFrame(user_data, columns=['id', 'name', 'full name', 'last_login',
                                          'projects'])
    display(df)     

In [8]:
from django.db import connection
from IPython.display import Markdown

def printmd(string):
    display(Markdown(string))


def get_user_ref_objects_count(
    cursor,
    user_id: int, table_name: str, field_names: List[str]) -> int:
    where_clause = ' or '.join([f'''"{f}" = {user_id}''' for f in field_names])
    cursor.execute(f'''select count(*) from "{table_name}" where {where_clause};''')
    return cursor.fetchone()[0]
    

def check_user_activity(user_id: int):
    u = User.objects.get(pk=user_id)
    print(f'#{user_id} {u.get_full_name()}, email: {u.email}.\nLast login: {u.last_login}')
    
    pms_count = len(u.get_all_permissions())
    groups = ', '.join(u.groups.values_list('name', flat=True)) or '-'
    print(f'User groups: {groups}, permissions total: {pms_count}')
    
    object_tables = [
        ('documents', 'document_document', ['modified_by_id', 'created_by_id']),
        ('document types', 'document_documenttype', ['created_by_id', 'modified_by_id']),
        ('document fields', 'document_documentfield', ['created_by_id', 'modified_by_id']),
        ('app. vars', 'common_appvar', ['user_id'])        
    ]
    with connection.cursor() as cursor:
        count_per_table = {title: get_user_ref_objects_count(cursor, user_id, tbl, cols) 
                           for title, tbl, cols in object_tables}

    counts = '\n'.join([f'{c}: {count_per_table[c]}' for c in count_per_table])
    if counts:
        printmd('The following objects reference the user #{user_id} and thus also **will be deleted**:')
        print(counts)   
        
    else:
        print("The user is not related to documents, types etc.")

In [13]:
from apps.rawdb.repository.raw_db_repository import RawDbRepository

def remove_user(user_id: int, save_mode = True):
    User.objects.filter(pk=user_id).delete()
    repo = RawDbRepository()
    repo.remove_user_references(user_id)
    print(f'User #{user_id} is removed')


<h1>Deleting a user</h1>

In the following cell we:
- [ optionally ] list ContraxSuite users
- [ optionally ] print user data
- delete the specific user

In [12]:
# - 1 - uncomment one of the following lines to list ContraxSuite users
#show_users_activity()  # list all CS users
#show_users_activity(sort_field='full name', sort_order='desc')  # ..., order by full name, desc
#show_users_activity(sort_field='full name')  # order by id, asc
#show_users_activity(id_filter=[1,2,3])  # list only users with id 1, 2 or 3
#show_users_activity(name='Ad')  # list only users with name (username or last_name or first_name) 
                                 # starting with 'Ad'
    
    

# - 2 - uncomment the following line to get summary on the user with provided id    
#check_user_activity(1)  


# - 3 - actually delete the user
#remove_user(61, True)  # safe mode (True) makes deleting slower but (in theory) prevents DB
                        # consistency issues  
remove_user(14)    

{"@timestamp": "2021-02-16T14:39:25.048161+00:00", "logger": "django", "level": "WARNING", "message": "_notify_field_value_saved failed because related object was probably deleted, original exception is \"Document matching query does not exist.\"", "special_log_type": "cx"}
{"@timestamp": "2021-02-16T14:39:25.055894+00:00", "logger": "django", "level": "WARNING", "message": "_notify_field_value_saved failed because related object was probably deleted, original exception is \"Document matching query does not exist.\"", "special_log_type": "cx"}
{"@timestamp": "2021-02-16T14:39:25.059046+00:00", "logger": "django", "level": "WARNING", "message": "_notify_field_value_saved failed because related object was probably deleted, original exception is \"Document matching query does not exist.\"", "special_log_type": "cx"}
{"@timestamp": "2021-02-16T14:39:25.062005+00:00", "logger": "django", "level": "WARNING", "message": "_notify_field_value_saved failed because related object was probably del